In [28]:
import boto3
from tqdm.notebook import tqdm
import pandas as pd

In [23]:
s3 = boto3.client('s3')

In [24]:
pag = s3.get_paginator('list_objects_v2')

In [25]:
page_iter = pag.paginate(Bucket='ft-modelling-lbeta', Prefix='cxr-nih/')

In [26]:
contents = []
for page in tqdm(page_iter):
    if 'Contents' in page:
        contents.extend(page['Contents'])
'Done'

'Done'

In [27]:
len(contents)

113178

In [29]:
df = pd.DataFrame.from_records(contents)

In [34]:
df['Filename'] = df['Key'].apply(lambda key: key.split('/')[-1])

In [57]:
test_filenames = []
with open('/Users/eoin/workspace/fourTheorem/lbeta/CADLab/CXR-Binary-Classifier/test_list.txt') as tlf:
    for line in tlf:
        test_filenames.append(line.strip())

In [58]:
df['IsTest'] = df['Filename'].apply(lambda fn: fn in test_filenames)

In [59]:
'00000003_000.png' in test_filenames

True

In [62]:
len(df[df.IsTest])

25763

In [65]:
df.head()

,Key,LastModified,ETag,Size,StorageClass,Filename,IsTest
0,cxr-nih/images_01/images/00000001_000.png,2020-11-14 21:40:50+00:00,"""9ed48cef82776548c0f4e074f94c11e5""",457171,STANDARD,00000001_000.png,False
1,cxr-nih/images_01/images/00000001_001.png,2020-11-14 21:40:50+00:00,"""96f0ea563a3c5814edc1f769cce5ae02""",444655,STANDARD,00000001_001.png,False
2,cxr-nih/images_01/images/00000001_002.png,2020-11-14 21:40:50+00:00,"""ce16f474cf68ee9e402ae25515255137""",351772,STANDARD,00000001_002.png,False
3,cxr-nih/images_01/images/00000002_000.png,2020-11-14 21:40:50+00:00,"""cd62c75e25b0969232ba67819e314332""",402332,STANDARD,00000002_000.png,False
4,cxr-nih/images_01/images/00000003_000.png,2020-11-14 21:40:50+00:00,"""b8fa05b9e43029ce6ac86f928026d739""",450000,STANDARD,00000003_000.png,True


In [66]:
df.to_csv('s3://ft-modelling-lbeta/cxr-nih/index.csv', header=True)